# CHECK COMPUTER AVAILABILITY

In [1]:
# Check GPU
!nvidia-smi

# Check CPU and RAM
!lscpu
!free -h

# Check disk space
!df -h

# Check details with psutil
import psutil

# CPU information
print("CPU:", psutil.cpu_percent(), "%")
print("Physical cores:", psutil.cpu_count(logical=False))
print("Logical cores:", psutil.cpu_count(logical=True))

# RAM information
mem = psutil.virtual_memory()
print("Total RAM:", mem.total / (1024 ** 3), "GB")
print("Available RAM:", mem.available / (1024 ** 3), "GB")

# Disk information
disk = psutil.disk_usage('/')
print("Total disk space:", disk.total / (1024 ** 3), "GB")
print("Used disk space:", disk.used / (1024 ** 3), "GB")
print("Free disk space:", disk.free / (1024 ** 3), "GB")

/bin/bash: line 1: nvidia-smi: command not found
Architecture:             x86_64
  CPU op-mode(s):         32-bit, 64-bit
  Address sizes:          46 bits physical, 48 bits virtual
  Byte Order:             Little Endian
CPU(s):                   2
  On-line CPU(s) list:    0,1
Vendor ID:                GenuineIntel
  Model name:             Intel(R) Xeon(R) CPU @ 2.20GHz
    CPU family:           6
    Model:                79
    Thread(s) per core:   2
    Core(s) per socket:   1
    Socket(s):            1
    Stepping:             0
    BogoMIPS:             4399.99
    Flags:                fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 cl
                          flush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc re
                          p_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3
                           fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand
                

# DOWNLOAD REQUERIMENTS



In [3]:
# prompt: install python requirements

!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.5/181.5 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.9/914.9 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 52.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-c

# EXECUTING CODE

In [4]:
import cv2
import numpy as np
import supervision as sv
from ultralytics import YOLO

input_video_path = '/content/demo.mp4'          # Reemplaza con tu ruta de video de entrada
output_video_path = '/content/output_video.mp4'   # Reemplaza con la ruta deseada para la salida

class CountObject():
    def __init__(self, input_video_path, output_video_path) -> None:
        self.model = YOLO('yolov8s.pt')
        # Agregué 4 colores para 4 zonas; puedes personalizarlos
        self.colors = sv.ColorPalette.from_hex(['#FF0000', '#00FF00', '#0000FF', '#FFFF00'])
        self.input_video_path = input_video_path
        self.output_video_path = output_video_path

        # Dimensiones del canvas (1920x1080, 16:9)
        width, height = 1920, 1080
        mid_x, mid_y = width // 2, height // 2

        # Definir 4 polígonos (cuadrantes) que juntos cubren exactamente el canvas
        self.polygons = [
            np.array([[0, 0], [mid_x, 0], [mid_x, mid_y], [0, mid_y]], np.int32),           # Zona superior izquierda
            np.array([[mid_x, 0], [width, 0], [width, mid_y], [mid_x, mid_y]], np.int32),       # Zona superior derecha
            np.array([[0, mid_y], [mid_x, mid_y], [mid_x, height], [0, height]], np.int32),       # Zona inferior izquierda
            np.array([[mid_x, mid_y], [width, mid_y], [width, height], [mid_x, height]], np.int32)  # Zona inferior derecha
        ]

        # Crear zonas a partir de los polígonos
        self.zones = [
            sv.PolygonZone(polygon=polygon)
            for polygon in self.polygons
        ]

        # Crear anotadores para cada zona
        self.zone_annotators = [
            sv.PolygonZoneAnnotator(
                zone=zone,
                color=self.colors.by_idx(index),
                thickness=6,
                text_thickness=8,
                text_scale=4
            )
            for index, zone in enumerate(self.zones)
        ]

        # Crear anotadores para las detecciones (cajas) correspondientes a cada zona
        self.box_annotators = [
            sv.BoxAnnotator(
                color=self.colors.by_idx(index),
                thickness=4,
            )
            for index in range(len(self.polygons))
        ]

    def process_frame(self, frame: np.ndarray, i) -> np.ndarray:
        # Realizar la detección de personas en el frame
        results = self.model(frame, imgsz=1280)[0]
        detections = sv.Detections.from_ultralytics(results)
        # Filtrar detecciones: clase 0 (personas) con confianza > 0.5
        detections = detections[(detections.class_id == 0) & (detections.confidence > 0.5)]

        # Recorrer cada zona para filtrar las detecciones y anotarlas
        for zone, zone_annotator, box_annotator in zip(self.zones, self.zone_annotators, self.box_annotators):
            mask = zone.trigger(detections=detections)
            detections_filtered = detections[mask]
            frame = box_annotator.annotate(scene=frame, detections=detections_filtered)
            frame = zone_annotator.annotate(scene=frame)

        return frame

    def process_video(self):
        sv.process_video(source_path=self.input_video_path,
                         target_path=self.output_video_path,
                         callback=self.process_frame)

if __name__ == "__main__":
    obj = CountObject(input_video_path, output_video_path)
    obj.process_video()


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 21.5M/21.5M [00:00<00:00, 164MB/s]



0: 736x1280 44 persons, 1 backpack, 4 handbags, 1 tennis racket, 1656.2ms
Speed: 38.5ms preprocess, 1656.2ms inference, 37.7ms postprocess per image at shape (1, 3, 736, 1280)

0: 736x1280 44 persons, 2 backpacks, 5 handbags, 1 tennis racket, 1402.8ms
Speed: 15.3ms preprocess, 1402.8ms inference, 2.9ms postprocess per image at shape (1, 3, 736, 1280)

0: 736x1280 42 persons, 1 backpack, 5 handbags, 1 tennis racket, 1409.4ms
Speed: 14.9ms preprocess, 1409.4ms inference, 3.7ms postprocess per image at shape (1, 3, 736, 1280)

0: 736x1280 43 persons, 6 backpacks, 4 handbags, 1 tennis racket, 1382.8ms
Speed: 14.0ms preprocess, 1382.8ms inference, 2.9ms postprocess per image at shape (1, 3, 736, 1280)

0: 736x1280 42 persons, 5 backpacks, 5 handbags, 1 tennis racket, 1399.1ms
Speed: 16.2ms preprocess, 1399.1ms inference, 2.7ms postprocess per image at shape (1, 3, 736, 1280)

0: 736x1280 41 persons, 4 backpacks, 6 handbags, 1 tennis racket, 2002.6ms
Speed: 14.4ms preprocess, 2002.6ms infer